# Main Library

In [1]:
import cv2, os
import numpy as np

In [2]:
face_cl = cv2.CascadeClassifier(r"D:\Pycharm\Computer Vision\haar cascade files-20230720T005324Z-001\haarcascade_frontalface_default.xml")
eye_cl = cv2.CascadeClassifier(r"D:\Pycharm\Computer Vision\haar cascade files-20230720T005324Z-001\eyes.xml")
mouth_cl = cv2.CascadeClassifier(r"D:\Pycharm\Computer Vision\haar cascade files-20230720T005324Z-001\mouth.xml")

glass = cv2.imread(r"D:\Pycharm\Computer Vision\haar cascade files-20230720T005324Z-001\glasses.png", cv2.IMREAD_UNCHANGED)
hat = cv2.imread(r"D:\Pycharm\Computer Vision\haar cascade files-20230720T005324Z-001\fedora.png", cv2.IMREAD_UNCHANGED)
cigar = cv2.imread(r"D:\Pycharm\Computer Vision\haar cascade files-20230720T005324Z-001\cigar.png", cv2.IMREAD_UNCHANGED)

In [3]:
def transparency(filter_img, frame):
    real_img = filter_img[:,:,0:3]
    alpha_img = filter_img[:,:,3]

    alpha_img = alpha_img[...,None]
    mask = alpha_img // 255

    result = mask * real_img +  frame * (1-mask)
    return result

def detect_object(filter_image, offset_x, offset_y, offset_w, offset_h,image, original, classifiar, color, color_Scale=1, min_N=5):
    original = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    result = classifiar.detectMultiScale(original, color_Scale, min_N)
    
    for (x, y, w, h) in result:
        if offset_x + x >= 0 and offset_y + y >= 0 and offset_w + w >= 0 and offset_h + h >= 0:
            x += offset_x
            y += offset_y
            w += offset_w
            h += offset_h
            if filter_image is None:
                cv2.rectangle(image, (x, y), (x+w, y+h), color, 1)
            else:
                filter_image = cv2.resize(filter_image, (w, h))
                clip = image[y:y+filter_image.shape[0], x:x+filter_image.shape[1]]
                
                result = transparency(filter_image, clip)
                image[y:y+filter_image.shape[0], x:x+filter_image.shape[1]] = result
        
    return image
        

In [6]:
Video = cv2.VideoCapture(0)
while True:
    _, frame = Video.read()
    
    
    result = detect_object(hat, 0, -120, 0, -50, frame, frame, face_cl, (0, 255, 0), 1.2, 5)
    result = detect_object(glass, 0, 0, 0, 0,result, frame, eye_cl, (255, 0, 0), 1.2, 4)
    result = detect_object(cigar, 50, -30, 0, 0,result, frame, mouth_cl, (0, 0, 255), 3.5, 7)
    
    cv2.imshow("image", result)
    if cv2.waitKey(1) == ord("o"):break

Video.release()
cv2.destroyAllWindows()

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4065: error: (-215:Assertion failed) inv_scale_x > 0 in function 'cv::resize'
